### Maven Airline Challenge

#####   Airline Passenger Satisfaction

#####   Recommended Analysis

1.  Which percentage of airline passengers are satisfied? Does it vary by customer type? What about type of travel?

2.  What is the customer profile for a repeating airline passenger?

3.  Does flight distance affect customer preferences or flight patterns?

4.  Which factors contribute to customer satisfaction the most? What about dissatisfaction?

In [16]:
#   import libraries

import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from zipfile import ZipFile

%matplotlib inline


In [17]:
'''
# Read zipfile

#   select filename
file_name = 'Airline+Passenger+Satisfaction.zip'

#   open file in read mode
with ZipFile(file_name, 'r') as zip:
    #   print the files in the zipfile
    zip.printdir()

    #   extract the files
    print('Extracting the files now. . .')
    zip.extractall()
    print('******DONE******')

'''

"\n# Read zipfile\n\n#   select filename\nfile_name = 'Airline+Passenger+Satisfaction.zip'\n\n#   open file in read mode\nwith ZipFile(file_name, 'r') as zip:\n    #   print the files in the zipfile\n    zip.printdir()\n\n    #   extract the files\n    print('Extracting the files now. . .')\n    zip.extractall()\n    print('******DONE******')\n\n"

####    Preview the data

In [18]:
#   Create dataframe to read first csv files
airline_df = pd.read_csv('airline_passenger_satisfaction.csv')
airline_df.head(5)

,ID,Gender,Age,Customer Type,Type of Travel,Class,Flight Distance,Departure Delay,Arrival Delay,Departure and Arrival Time Convenience,...,On-board Service,Seat Comfort,Leg Room Service,Cleanliness,Food and Drink,In-flight Service,In-flight Wifi Service,In-flight Entertainment,Baggage Handling,Satisfaction
0,1,Male,48,First-time,Business,Business,821,2,5.0,3,...,3,5,2,5,5,5,3,5,5,Neutral or Dissatisfied
1,2,Female,35,Returning,Business,Business,821,26,39.0,2,...,5,4,5,5,3,5,2,5,5,Satisfied
2,3,Male,41,Returning,Business,Business,853,0,0.0,4,...,3,5,3,5,5,3,4,3,3,Satisfied
3,4,Male,50,Returning,Business,Business,1905,0,0.0,2,...,5,5,5,4,4,5,2,5,5,Satisfied
4,5,Female,49,Returning,Business,Business,3470,0,1.0,3,...,3,4,4,5,4,3,3,3,3,Satisfied


In [19]:
#   Create dataframe to read second csv file
data_df = pd.read_csv('data_dictionary.csv')
data_df

,Field,Description
0,ID,Unique passenger identifier
1,Gender,Gender of the passenger (Female/Male)
2,Age,Age of the passenger
3,Customer Type,Type of airline customer (First-time/Returning)
4,Type of Travel,Purpose of the flight (Business/Personal)
5,Class,Travel class in the airplane for the passenger...
6,Flight Distance,Flight distance in miles
7,Departure Delay,Flight departure delay in minutes
8,Arrival Delay,Flight arrival delay in minutes
9,Departure and Arrival Time Convenience,Satisfaction level with the convenience of the...


In [20]:
#   Check total count of rows and columns
airline_df.shape

(129880, 24)

In [21]:
#   Check info
airline_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129880 entries, 0 to 129879
Data columns (total 24 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   ID                                      129880 non-null  int64  
 1   Gender                                  129880 non-null  object 
 2   Age                                     129880 non-null  int64  
 3   Customer Type                           129880 non-null  object 
 4   Type of Travel                          129880 non-null  object 
 5   Class                                   129880 non-null  object 
 6   Flight Distance                         129880 non-null  int64  
 7   Departure Delay                         129880 non-null  int64  
 8   Arrival Delay                           129487 non-null  float64
 9   Departure and Arrival Time Convenience  129880 non-null  int64  
 10  Ease of Online Booking                  1298

In [22]:
#   Check for duplicates
airline_df.duplicated().sum()

#duplicates = airline_df[airline_df.duplicated()]
#duplicates

0

####    Data Cleaning

In [23]:
#   Review the column names
airline_df.columns

Index(['ID', 'Gender', 'Age', 'Customer Type', 'Type of Travel', 'Class',
       'Flight Distance', 'Departure Delay', 'Arrival Delay',
       'Departure and Arrival Time Convenience', 'Ease of Online Booking',
       'Check-in Service', 'Online Boarding', 'Gate Location',
       'On-board Service', 'Seat Comfort', 'Leg Room Service', 'Cleanliness',
       'Food and Drink', 'In-flight Service', 'In-flight Wifi Service',
       'In-flight Entertainment', 'Baggage Handling', 'Satisfaction'],
      dtype='object')

In [24]:
#   Rename the column names to remove spaces and replace with underscore and also change them to lower case
def change(new):
    airline_df.columns = airline_df.columns.str.replace(" ","_", regex=True)
    airline_df.columns = airline_df.columns.str.lower()
    return airline_df.columns

change(airline_df)

Index(['id', 'gender', 'age', 'customer_type', 'type_of_travel', 'class',
       'flight_distance', 'departure_delay', 'arrival_delay',
       'departure_and_arrival_time_convenience', 'ease_of_online_booking',
       'check-in_service', 'online_boarding', 'gate_location',
       'on-board_service', 'seat_comfort', 'leg_room_service', 'cleanliness',
       'food_and_drink', 'in-flight_service', 'in-flight_wifi_service',
       'in-flight_entertainment', 'baggage_handling', 'satisfaction'],
      dtype='object')

In [25]:
#   Check for null values in the columns
airline_df.isnull().any()

id                                        False
gender                                    False
age                                       False
customer_type                             False
type_of_travel                            False
class                                     False
flight_distance                           False
departure_delay                           False
arrival_delay                              True
departure_and_arrival_time_convenience    False
ease_of_online_booking                    False
check-in_service                          False
online_boarding                           False
gate_location                             False
on-board_service                          False
seat_comfort                              False
leg_room_service                          False
cleanliness                               False
food_and_drink                            False
in-flight_service                         False
in-flight_wifi_service                  

####    Save the dataframe as CSV to allow visualization in Tableau

In [28]:
# Save the dataframe as csv
airline_df.to_csv('airline_new.csv', index = False, encoding = 'utf-8')
print (airline_df)

            id  gender  age customer_type type_of_travel         class  \
0            1    Male   48    First-time       Business      Business   
1            2  Female   35     Returning       Business      Business   
2            3    Male   41     Returning       Business      Business   
3            4    Male   50     Returning       Business      Business   
4            5  Female   49     Returning       Business      Business   
...        ...     ...  ...           ...            ...           ...   
129875  129876    Male   28     Returning       Personal  Economy Plus   
129876  129877    Male   41     Returning       Personal  Economy Plus   
129877  129878    Male   42     Returning       Personal  Economy Plus   
129878  129879    Male   50     Returning       Personal  Economy Plus   
129879  129880  Female   20     Returning       Personal  Economy Plus   

        flight_distance  departure_delay  arrival_delay  \
0                   821                2            